In [ ]:
%load_ext autoreload
%autoreload 2

# Distributions of parameters

For every field that we want to use as input for deep learning, plot their distributions, so that we know what kind of normalization to apply.
I expect that we will mostly have normal distritbutions (or use normal distritbutions even though we shouldn't).

## Boot dask cluster

In [ ]:
import dask
import dask.array as da
import dask.distributed
import dask_jobqueue
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import scipy.stats
import xarray as xr
import xskillscore as xs

In [ ]:
from crims2s.util import fix_dataset_dims

In [ ]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
INPUT_PROCESSED = '***BASEDIR***processed/training-input/'
OBSERVATIONS = '***BASEDIR***training-output-reference/'
BENCHNMARK = '***BASEDIR***training-output-benchmark/'
CENTER = 'ecmwf'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)
obs_path = pathlib.Path(OBSERVATIONS)
processed_path = pathlib.Path(INPUT_PROCESSED)

## Boost dask cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
)

In [ ]:
cluster.scale(jobs=6)  # Scale to two working nodes as configured.

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

## Surface temperature

In [ ]:
FIELD = 't2m'

In [ ]:
field_files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])
ecmwf = xr.open_mfdataset(field_files, preprocess=fix_dataset_dims)
ecmwf = ecmwf.persist()

In [ ]:
### Overall mean

In [ ]:
t2m_mean = ecmwf.t2m.mean().compute()

In [ ]:
t2m_std = ecmwf.t2m.std().compute()

In [ ]:
t2m_mean

In [ ]:
t2m_std

### Check shape of distribution

In [ ]:
ecmwf.t2m.isel(forecast_year=10).plot.hist()

This is not as gaussian as I'd like but for now a normal will do.

### Check yearly progression

Should we adjust for climate change correlation?
Ideally we would do this on a tile by tile basis (and look for a correlation between year and average temperature).
If there is a strong correlation, one way or the other, we should adjust for temperature.

In [ ]:
ecmwf.dims

In [ ]:
yearly_means = ecmwf.t2m.mean(dim=['realization', 'forecast_dayofyear', 'lead_time', 'latitude', 'longitude']).compute()

In [ ]:
yearly_means.plot()

The trend is pretty clear! But it's about 1 degree over our training dataset. Given that temperture had a std of about 20, i'm not sure it matters enough to be priority.

Also, our models will be boosting models, so if the model incorporates this trend there could be double accounting.

In [ ]:
obs_files = [f for f in obs_path.iterdir() if FIELD in f.stem]
obs = xr.open_mfdataset(obs_files, preprocess=fix_dataset_dims)

In [ ]:
obs_yearly_means = obs.t2m.mean(dim=['forecast_dayofyear', 'lead_time', 'latitude', 'longitude']).compute()

In [ ]:
obs_yearly_means.plot()

In [ ]:
del ecmwf

In [ ]:
del obs

## Other temperatures

In [ ]:
FIELD = '-t-'
field_files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
field_files[:10]

In [ ]:
ecmwf = fix_dataset_dims(xr.open_dataset(field_files[0], chunks={'plev': 1}))

In [ ]:
ecmwf

In [ ]:
ecmwf = xr.open_mfdataset(field_files, preprocess=fix_dataset_dims, chunks={'plev': 1})


In [ ]:
ecmwf

In [ ]:
by_plev = ecmwf.mean(dim=['latitude', 'longitude']).persist()

In [ ]:
by_plev

In [ ]:
del by_plev

In [ ]:
c = by_plev.compute()
c

In [ ]:
ecmwf = ecmwf.persist()

In [ ]:
ecmwf

In [ ]:
dask.config.get('array.chunk-size')

In [ ]:
t_mean = ecmwf.mean().compute()
t_mean

In [ ]:
t_std = ecmwf.mean().compute()
t_std

## Precipitation

For precipitation we will do our lienar rescaling over 2 weeks since the provided data is accumulated precipitation.
To the mean will be mean total precipitation over 2 weeks, etc.

In [ ]:
FIELD = 'tp'

In [ ]:
field_files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and f'-{FIELD}-' in f.stem])

In [ ]:
field_files[:10]

In [ ]:
ecmwf = xr.open_mfdataset(field_files, preprocess=fix_dataset_dims)
ecmwf = ecmwf.persist()

In [ ]:
ecmwf

In [ ]:
w12 = ecmwf.isel(lead_time=14).drop('valid_time')
w34 = ecmwf.isel(lead_time=28) - w12
w56 = ecmwf.isel(lead_time=42) - ecmwf.isel(lead_time=28)

In [ ]:
w12

In [ ]:
w34

In [ ]:
leads = [datetime.timedelta(days=x) for x in [14, 28, 42]]

In [ ]:
tp = xr.concat([w12, w34, w56], dim='lead_time').assign_coords(lead_time=xr.DataArray(data=leads, dims='lead_time'))

In [ ]:
tp.mean(dim=['latitude', 'longitude', 'realization', 'forecast_dayofyear', 'forecast_year']).compute()

In [ ]:
tp_mean = tp.mean().compute()
tp_mean

In [ ]:
tp_std = tp.std().compute()
tp_std

In [ ]:
### Overall mean

In [ ]:
tp_mean = ecmwf.tp.mean().compute()
tp_mean

In [ ]:
tp_std = ecmwf.tp.std().compute()
tp_std

In [ ]:
ecmwf

In [ ]:
ecmwf.isel(forecast_dayofyear=0, forecast_year=0, latitude=40, longitude=10).mean(dim=['realization']).tp.plot()

## Geopotential Height

We read this field from the processed dir instead of the input dir.
Consequently there is no need for fix_dataset_dims we already did that when procesing the file.

In [ ]:
HEIGHTS = [100]

In [ ]:
for h in HEIGHTS:
    FIELD = f'-gh{h}-'
    field_files = sorted([f for f in processed_path.iterdir() if CENTER in f.stem and FIELD in f.stem])
    print(f'{len(field_files)} files.')
    ecmwf = xr.open_mfdataset(field_files)
    ecmwf = ecmwf.persist()
    mean = ecmwf.mean().compute().gh.item()
    std = ecmwf.std().compute().gh.item()
    del ecmwf
    print(f'GH{h}. Mean: {mean}. STD: {std}.')

In [ ]:
field_files = sorted([f for f in processed_path.iterdir() if CENTER in f.stem and FIELD in f.stem])
ecmwf = xr.open_mfdataset(field_files)

In [ ]:
ecmwf.isel(lead_time=0).gh.plot()

In [ ]:
client.restart()

## Sea Surface Temperature

In [ ]:
client.restart()

In [ ]:
FIELD = 'sst'

In [ ]:
field_files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])
ecmwf = xr.open_mfdataset(field_files, preprocess=fix_dataset_dims)
ecmwf = ecmwf.persist()

In [ ]:
### Overall mean

In [ ]:
sst_mean = ecmwf.sst.mean().compute()
sst_mean

In [ ]:
sst_std = ecmwf.sst.std().compute()
sst_std